In [ ]:
import requests
import json
import os
import pandas as pd
import csv
from bs4 import BeautifulSoup
import numpy as np
import time

針對 高雄公有蔬果零售價格進行查詢及爬取
https://96kuas.kcg.gov.tw/prices/market.qd1.php

原始資訊取得

In [ ]:
def url_get(year, month, day):
    headers = {'cookie':'Test', 
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0'}
    url = 'https://96kuas.kcg.gov.tw/prices/market.qd1.php'
    params={'tclassSel':"0全部資料",
            'yy':f'{year}',
            'mm':f'{month}',
            'dd':f'{day}',
            'subm':"開始查詢"}
    resp = requests.post(url, headers=headers, params=params)
    resp.encoding = 'utf-8'
    data = BeautifulSoup(resp.text, 'html.parser')
    date_time = f'{year+1911}-{month}-{day}'
    return data, date_time

html解碼

In [ ]:
def html_decode():
    a = data.body.div.find_all('div')[1]
    b = a.find_all('table')[1]
    main_inf = b.find_all('tr') # list 型態  從[2]開始有蔬菜到[??]有夾雜其他欄位資訊 要剔除
    return main_inf

由html解碼後的資訊中取得 蔬果價格列表

In [ ]:
def market_name():
    market_name_list = []
    for i in range(1, len(main_inf[0].find_all('td'))):
        market_name_list.append(main_inf[0].find_all('td')[i].contents[0])
    return market_name_list
def columns_name():
    market_name_list = market_name()
    columns_list = market_name_list
    columns_list.insert(0, '日期')
    return columns_list

整理蔬果列表後  存為np 

In [ ]:
def item_extract(n):
    item_names = []
    price_lists = []
    pl = []
    for i in range(len(main_inf[n].find_all('td'))):
        if i == 0:
            item_names.append(main_inf[n].find_all('td')[i].contents)
        else:
            price_lists.append(main_inf[n].find_all('td')[i].contents)
    for i in price_lists:
        if i == []:
            i = None
            pl.append(i)
        else:
            i = i[0] 
            pl.append(i)
    price_lists = pl
    price_lists.insert(0, date_time)
    return item_names, price_lists

def inf_get():
    item = []
    price = []

    for n in range(2, len(main_inf)):
        if  main_inf[n].find_all('td')[0].contents[0].find('\n') != -1:
            continue
        item_names, price_lists = item_extract(n)
        item.append(item_names)
        price.append(price_lists)
    return item, price

def set_table():
    item, price= inf_get()
    item = np.array(item).reshape(-1)
    market_df = pd.DataFrame(price)
    market_df.columns = columns_name()
    market_df.insert(0, '品項', pd.DataFrame(item))
    retail_market_1day = np.array(market_df)    
    return retail_market_1day, market_df


# 實際執行程式

In [ ]:
# 先存入一日之資訊，做為之後爬蟲資料儲存的參數
data, date_time = url_get(107, 12, 31)
main_inf = html_decode()
retail_market_1day, market_df = set_table()
retail_market = np.array([])

# 將109-110年的網站資訊全數抓下
for y in range(108, 109): 
    for m in range(1, 13): # 1-12月
        if m in [1, 3, 5, 7, 8, 10, 12]:
            md = 31
        elif m in [4, 6, 9, 11]: 
            md = 30
        else: # 閏年29 平年28
            if (y+1911)%4 == 0:
                md = 29
            else:
                md = 28
                        
        for d in range(1, md+1):  
            data, date_time = url_get(y, m, d)
            main_inf = html_decode()

            if retail_market.shape[0] < 252:
                retail_market = np.concatenate([retail_market_1day, set_table()[0]])  
            else:
                retail_market = np.concatenate([retail_market, set_table()[0]])  
            time.sleep(1)
retail_market.shape

(46116, 17)

In [ ]:
with open('market_menu.csv', 'w', encoding='utf-8', newline='') as f1:
    writer = csv.writer(f1)
    writer.writerow(list(market_df.columns))
    for i in retail_market:
        writer.writerow(i)
